# DART Open API를 이용해 기업 공시 정보를 가져온다. 

DART Open API 사용 연습

해당 코드는 Python 3.5/3.7, 32bit/64bit 상관 없다. 

In [2]:
# -*- coding: utf-8 -*-

from urllib.request import Request
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

import re
import json
import xml.etree.ElementTree as elemTree
import sys

## DART Open API와 연결

DART API에는 4가지 정보가 있고, 각 정보는 더 세부적으로 나뉜다. 
1. 공시정보
2. 사업보고서 주요정보
3. 상장기업 재무정보
4. 지분공시 종합정보

In [3]:
with open('./DART_password.txt', 'r') as f:
    API_KEY = f.read()

In [6]:
crtfc_key = '?crtfc_key=' + API_KEY

1. 공시정보: 
    - 공시검색: 공시 유형별, 회사별, 날짜별 등 여러가지 조건으로 공시보고서 검색기능을 제공합니다.
    - 기업개황: DART에 등록되어있는 기업의 개황정보를 제공합니다.
    - 공시서류원본파일: 공시보고서 원본파일을 제공합니다.
    - 고유번호: DART에 등록되어있는 공시대상회사의 고유번호,회사명,대표자명,종목코드, 최근변경일자를 파일로 제공합니다.

기타 세부사항은 API doc에서 확인: https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019001

In [7]:
## 공시정보 base URLs

DART_list_json = 'https://opendart.fss.or.kr/api/list.json' # 공시검색
DART_company_json = 'https://opendart.fss.or.kr/api/company.json' # 기업개황
DART_document_xml = 'https://opendart.fss.or.kr/api/document.xml' # 공시서류원본파일
DART_corpCode_xml = 'https://opendart.fss.or.kr/api/corpCode.xml' # 고유번호

In [8]:
def to_parameter(key, value):
    return '&' + str(key) + '=' + str(value)

In [9]:
def DART_annc_info(info_type, **kwargs):
    parameters = ''
    for k, v in kwargs.items():
        parameters += to_parameter(k,v)
    
    if info_type == 'list':
        return DART_list_json + crtfc_key + parameters
    elif info_type == 'company':
        return DART_company_json + crtfc_key + parameters
    elif info_type == 'document':
        return DART_document_xml + crtfc_key + parameters
    elif info_type == 'corpCode':
        return DART_corpCode_xml + crtfc_key + parameters
    else:
        print('Wrong info_type. Choose from:')
        print('''
        1. "list": 공시검색
        2. "company": 기업개활
        3. "document": 공시서류원본파일
        4. "corpCode": 고유번호
        ''')

In [10]:
def DART_get_response(request_url):
    req = urlopen(request_url)
    response = req.read().decode('utf8')
    
    try:
        result = ('json', json.loads(response))
    except JSONDecodeError:
        result = ('xml', elemTree.fromstring(response))
    except:
        print("An error occurred: ", sys.exc_info()[0])
        return 0
        
    return result

In [11]:
req_url = DART_annc_info('list', corp_code='00919966', bgn_de='20190801', end_de='20190815')
req_url

'https://opendart.fss.or.kr/api/list.json?crtfc_key=407c1fe7fc7a1a183002c6d5f981408662cd879e&corp_code=00919966&bgn_de=20190801&end_de=20190815'

In [12]:
DART_get_response(req_url)

('json',
 {'status': '000',
  'message': '정상',
  'page_no': 1,
  'page_count': 10,
  'total_count': 5,
  'total_page': 1,
  'list': [{'corp_code': '00919966',
    'corp_name': '신라젠',
    'stock_code': '215600',
    'corp_cls': 'K',
    'report_nm': '반기보고서 (2019.06)',
    'rcept_no': '20190814002650',
    'flr_nm': '신라젠',
    'rcept_dt': '20190814',
    'rm': ''},
   {'corp_code': '00919966',
    'corp_name': '신라젠',
    'stock_code': '215600',
    'corp_cls': 'K',
    'report_nm': '임원ㆍ주요주주특정증권등소유상황보고서',
    'rcept_no': '20190809000049',
    'flr_nm': '송명석',
    'rcept_dt': '20190809',
    'rm': ''},
   {'corp_code': '00919966',
    'corp_name': '신라젠',
    'stock_code': '215600',
    'corp_cls': 'K',
    'report_nm': '임원ㆍ주요주주특정증권등소유상황보고서',
    'rcept_no': '20190809000045',
    'flr_nm': '권혁찬',
    'rcept_dt': '20190809',
    'rm': ''},
   {'corp_code': '00919966',
    'corp_name': '신라젠',
    'stock_code': '215600',
    'corp_cls': 'K',
    'report_nm': '임원ㆍ주요주주특정증권등소유상황보고서',
    'rcept_n

## 공시시간 크롤링

문의결과, DART API는 현재 공시시간 정보를 제공하지 않는다. (아직 시범운영기간임을 감안하긴 해야한다.)

따라서, 최근 공시 페이지는 직접 크롤링하기로 하였다. 다소 번거롭지만 API에서 지원이 되기 전까진 DART API에서 기본적인 공시 정보를 가져오고, 분단위의 공시시간이 필요한 경우 해당 날짜의 최근 공시를 크롤링한 결과와 대조해 결과를 매칭시키도록 한다. 

<font style='color:red;'>TODO: 여러 날짜의 DailyAnnc 를 모아야한다. Class화 하는 것은 어떨까? 하지만 Class화 할 이유가 있을까? 고민해보자. </font>

In [16]:
def get_recent_annc_list_bs(date):
    date_regex = re.compile(r'^\d{4}\.\d{2}\.\d{2}$')
    if not date_regex.match(date):
        print("Error: Date format should be - yyyy.mm.dd")
        return 0
    
    recent_annc_url = 'http://dart.fss.or.kr/dsac001/mainK.do?selectDate={date}&sort=&series=&mdayCnt=0#'.format(date=date)
    recent_annc_req = urlopen(recent_annc_url)
    recent_annc_bs = bs(recent_annc_req, 'html.parser')
    
    recent_annc_list_bs = recent_annc_bs.select('div.table_list > table > tr')
    
    return recent_annc_list_bs

재밌는 사실. 이 소스코드엔 애초에 tbody가 없었다. 그래서 한참 헤맸다. 

Chrome selector에선 #listContents > div.table_list > table > tbody > tr:nth-child(1) 라고 나왔지만 이는 브라우저가 insert시킨 것이다. 이런 점을 고려하여 selector를 써야한다. 무조건 Chrome에서 copy selector 한다고 되지 않는다. 

관련: https://stackoverflow.com/questions/20522820/how-to-get-tbody-from-table-from-python-beautiful-soup

In [17]:
def annc_bs2data(annc_bs):
    annc_time_regex = re.compile(r'\d\d:\d\d')
    annc_time = annc_bs.find('td', attrs={'class':'cen_txt'}).text
    annc_time = annc_time_regex.search(annc_time).group()
    
    corp_code_regex = re.compile(r'\d{8}')
    corp_code = annc_bs.find('span', {'class':'nobr1'}).a.attrs['onclick']
    corp_code = corp_code_regex.search(corp_code).group()
    
    annc_content_regex = re.compile(r'openReportViewer')
    annc_content = annc_bs.find('a', attrs={'onclick':annc_content_regex})  
    
    annc_title_regex = re.compile('\\\\.')
    annc_content_text = annc_content.text 
    str_text = "%r"%annc_content_text # raw string으로 변환시켜줘야 \t가 tab으로 인식되지 않는다. 
    raw_text = str_text[1:-1]
    
    annc_title = re.sub(annc_title_regex, '', raw_text).strip() 
    
    annc_id_regex = re.compile(r'\d+')
    annc_id = annc_id_regex.search(annc_content.attrs['id']).group()
    
    data = {}
    data['annc_time'] = annc_time
    data['corp_code'] = corp_code
    data['annc_title'] = annc_title
    data['annc_id'] = annc_id
    
    return data

<font style='color:red'>TODO: raw string으로 바꾸는 부분, 전혀 원리를 모르겠다. </font>

In [20]:
annc_20190802 = get_recent_annc_list_bs('2019.08.01')

for i in annc_20190802:
    print(annc_bs2data(i))
    print("---"*5)

{'annc_time': '18:22', 'corp_code': '00442455', 'annc_title': '전환가액의조정  (제10,11,12,13,14,15회차)', 'annc_id': '20190801900579'}
---------------
{'annc_time': '18:08', 'corp_code': '00122339', 'annc_title': '대표집행임원변경', 'annc_id': '20190801900573'}
---------------
{'annc_time': '18:08', 'corp_code': '00122339', 'annc_title': '대표집행임원변경', 'annc_id': '20190801900568'}
---------------
{'annc_time': '18:03', 'corp_code': '00957568', 'annc_title': '최대주주변경', 'annc_id': '20190801900550'}
---------------
{'annc_time': '18:03', 'corp_code': '00244419', 'annc_title': '[기재정정]주요사항보고서(유형자산양수결정)', 'annc_id': '20190801000442'}
---------------
{'annc_time': '18:00', 'corp_code': '00541437', 'annc_title': '유상증자또는주식관련사채등의청약결과(자율공시)', 'annc_id': '20190801900555'}
---------------
{'annc_time': '18:00', 'corp_code': '00298687', 'annc_title': '주권매매거래정지기간변경  (상장적격성 실질심사 대상 (사유발생))', 'annc_id': '20190801900554'}
---------------
{'annc_time': '17:54', 'corp_code': '00122339', 'annc_title': '[기재정정]주요사항보고서(유상증자결정)', 